In [1]:
# importing the lib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import time
import sys
from selenium.webdriver.chrome.service import service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import chromedriver_binary
import re
import numpy as np

In [2]:
def kayak(doj,souce_airport,destination_airport):
    url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    time.sleep(25)
    def click_more():
    
        for val in range(0,50):
            try:
                driver.find_element_by_xpath("//a[@class='moreButton']").click()
                time.sleep(5)
                val = val +1

            except:
                pass
    click_more()
    
    # depart-time base-time
    departure_time=[]
    departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

    arrival_time=[]
    arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

    duration =[]
    duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")


    no_stop = []
    no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

    price =[]
    price = driver.find_elements_by_xpath("//div[@class = 'booking']")

    airlines = []
    airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

    # departure_time = driver.find_elements_by_xpath("//div[@class='itinerary-depart-time depart-time']")

    
    
    
    price_list=[]
    stops=[]
    for i in range(len(departure_time)):
        departure_time[i]= departure_time[i].get_attribute('innerHTML')
        arrival_time[i] = arrival_time[i].get_attribute('innerHTML')
        duration[i] = duration[i].get_attribute('innerHTML')
        no_stop[i] = no_stop[i].get_attribute('innerHTML')
        l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
        stops.append(l)
        price[i] = price[i].get_attribute('innerHTML')

#         airlines[i] = airlines[i].get_attribute('innerHTML')
        b = re.findall("nbsp\;((\d\d\d)|(\d.\d\d\d)|(\d\d.\d\d\d))",price[i])[0][0]
        price_list.append(b)
    
    for k in range(len(airlines)):
        airlines[k] = airlines[k].get_attribute('innerHTML')
        
    # Creating data frames
    departure = len(departure_time)*[str(source_airport)]
    arrival = len(departure_time)*[str(destination_airport)]
    # a = pd.DataFrame(departure_time,arrival_time,price_list)
    doj1 = len(departure_time)*[doj]
    df  = pd.DataFrame(doj1,columns=["Date of journey"])
    df["Source"] = departure    
    df["Destination"] = arrival
    df["Departure_time"] = departure_time
    df["Arrival_time"] = arrival_time
    df["Duration"] = duration
    df["Stops"] = stops
    df["CAD $"]= price_list
    df["Airlines"] = airlines
#     df["Stops"]=df["Stops"].astype(str).str.replace("\n","")
    df["CAD $"]=df["CAD $"].astype(str).str.replace(",","")
    df["CAD $"] = pd.to_numeric(df['CAD $'])
#     df["Stops"]=df["Stops"].astype(str).str.replace("stops","")
#     df["Stops"]=df["Stops"].astype(str).str.replace("stop","")
    df["Duration"]=df["Duration"].astype(str).str.replace("\n","")
    df = df.drop(df[(df.Stops =='3') | (df.Airlines == 'Multiple Airlines')].index)
    df= df.drop_duplicates()
    df = df.sort_values(by='CAD $')
    df = df.reset_index()
    df.index = df.index +1
    df.pop("index")
    driver.close()
    return df
    
    

In [3]:
source = ["DEL"]
destination =  ["YYZ"]
# source = ["YVR"]
# destination = ["BOM","AMD"]

doj=['2022-05-30']
df1 = pd.DataFrame(columns=['Date of journey', 'Source', 'Destination', 'Departure_time',
                            'Arrival_time', 'Duration', 'Stops', 'CAD $', 'Airlines'])
for val in source:
    for values in destination:
        source_airport = val
        print(source_airport)
        
        
        destination_airport = values
        print(destination_airport)
        doj ='2022-05-30'
        df = kayak(doj,source_airport,destination_airport)
#         print("Departure length",len(departure_time))
#         print(df1.describe())
        df.to_csv('kayak1_data.csv', mode='a',index=False,header = False)
#         df1=pd.concat([df1,df])
#         print(df1.shape())
        

DEL
YYZ


C:\Users\HETSHAH\AppData\Local\Temp/ipykernel_29160/743859376.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()
C:\Users\HETSHAH\AppData\Local\Temp/ipykernel_29160/743859376.py:21: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")
C:\Users\HETSHAH\AppData\Local\Temp/ipykernel_29160/743859376.py:24: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")
C:\Users\HETSHAH\AppData\Local\Temp/ipykernel_29160/743859376.py:27: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by

In [4]:
df

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,DEL,YYZ,12:35 a.m.,2:01 p.m.,46h 56m,2,957,Delta
2,2022-05-30,DEL,YYZ,11:35 p.m.,3:20 p.m.,25h 15m,1,1246,United Airlines
3,2022-05-30,DEL,YYZ,1:55 a.m.,1:21 p.m.,20h 56m,1,1246,United Airlines
4,2022-05-30,DEL,YYZ,11:35 p.m.,1:09 p.m.,23h 04m,1,1246,United Airlines
5,2022-05-30,DEL,YYZ,8:00 a.m.,8:15 p.m.,21h 45m,1,1386,LOT
...,...,...,...,...,...,...,...,...,...
100,2022-05-30,DEL,YYZ,5:25 a.m.,8:45 p.m.,24h 50m,1,7674,"Air India, Lufthansa"
101,2022-05-30,DEL,YYZ,5:25 a.m.,6:55 p.m.,23h 00m,2,7920,"Air India, Lufthansa"
102,2022-05-30,DEL,YYZ,5:25 a.m.,9:32 p.m.,25h 37m,2,7920,"Air India, Lufthansa"
103,2022-05-30,DEL,YYZ,2:50 a.m.,3:55 p.m.,22h 35m,2,7969,"Lufthansa, British Airways"


# DONT RUN BELOW BLOCKS>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
source_airport="DEL"
destination_airport="YHZ"
doj='2022-05-30'
url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)
time.sleep(15)
def click_more():

    for val in range(0,50):
        try:
            driver.find_element_by_xpath("//a[@class='moreButton']").click()
            time.sleep(4)
            val = val +1

        except:
            pass
click_more()

In [ ]:
departure_time=[]
departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

arrival_time=[]
arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

duration =[]
duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")

no_stop = []
no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

price =[]
price = driver.find_elements_by_xpath("//div[@class = 'booking']")

airlines = []
airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

In [ ]:
len(no_stop)

In [ ]:
stops=[]

In [ ]:
type(no_stop[1])

In [ ]:
for i in range(len(departure_time)):
    no_stop[i] = no_stop[i].get_attribute('innerHTML')
    l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
    stops.append(l)

In [ ]:
stops